In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X.shape, y.shape

((1000, 10), (1000,))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((700, 10), (700,)), ((300, 10), (300,)))

In [5]:
lr = LogisticRegression(C=1, solver='liblinear')
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [6]:
rf = RandomForestClassifier(n_estimators=45, max_depth=3)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



In [7]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=34)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(X_train_res, return_counts=True)

(array([-5.51207083, -5.14120195, -4.69421946, ...,  5.71550663,
         5.79413843,  6.02415826]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [9]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train_res, y_train_res)
preds = xgb.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       270
           1       0.76      0.87      0.81        30

    accuracy                           0.96       300
   macro avg       0.87      0.92      0.90       300
weighted avg       0.96      0.96      0.96       300



## Experiments tracking with MLflow

In [10]:
models = [
    (
        'Logistic Regression',
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'Random Forest',
        RandomForestClassifier(n_estimators=45, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'XGB 1',
        XGBClassifier(eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'XGB 2',
        XGBClassifier(eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [11]:
metrics = {}

for i, mod in enumerate(models):
    model_name = mod[0]
    model = mod[1]
    model.fit(mod[2][0], mod[2][1])
    preds = model.predict(mod[3][0])
    report = classification_report(mod[3][1], preds, output_dict=True)
    metrics[i] = report

metrics

{0: {'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669724,
   'support': 270},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.9132610508757297,
   'support': 300}},
 1: {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300},
  'weighted avg': {'precision'

# Dagshub

In [12]:
import dagshub
dagshub.init(repo_owner='Harsh-Agarwals', repo_name='Anomaly-Detection-MLflow', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=dc98117a-fc14-4b2e-8736-bf5fafd23b9f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c9cf92a2069bfe688e07f7384e5b95310106d9ebb57087f992276efc75f229cb




Accessing as Harsh-Agarwals

Initialized MLflow to track repo "Harsh-Agarwals/Anomaly-Detection-MLflow"

Repository Harsh-Agarwals/Anomaly-Detection-MLflow initialized!

In [13]:
import mlflow
import os

# Uncomment this if code does not works
# os.environ['MLFLOW_TRACKING_USERNAME'] = "Harsh-Agarwals"
# os.environ['MLFLOW_TRACKING_PASSWORD'] = "a062057ec939f630271d5fbcfa47c8f0ddbe59b1"
# os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/Harsh-Agarwals/Anomaly-Detection-MLflow.mlflow"


mlflow.set_experiment("multiple_models")
# mlflow.set_registry_uri(uri="http://127.0.0.1:5000/")
mlflow.set_registry_uri("https://dagshub.com/Harsh-Agarwals/Anomaly-Detection-MLflow.mlflow")

for i, model in enumerate(models):
    model_name = model[0]
    model = model[1]
    report = metrics[i]
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param('model', model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_0', report['0']['recall'])
        mlflow.log_metric('recall_1', report['1']['recall'])
        mlflow.log_metric('f1-score', report['macro avg']['f1-score'])
        if 'xgb' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2024/08/19 23:41:25 INFO mlflow.tracking.fluent: Experiment with name 'multiple_models' does not exist. Creating a new experiment.
2024/08/19 23:41:40 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/19 23:41:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/Harsh-Agarwals/Anomaly-Detection-MLflow.mlflow/#/experiments/0/runs/de3ba820d0d94960879702e3749f2750.
2024/08/19 23:41:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Harsh-Agarwals/Anomaly-Detection-MLflow.mlflow/#/experiments/0.
2024/08/19 23:41:49 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/19 23:41:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/Harsh-Agarwals/A

## Model Registry

In [21]:
model_name = "XGB_SMOTE"
run_ID = input("Please type run ID: ") # d88b94bf2fc347e1a4e3f2849e787c05
model_uri = f'runs:/{run_ID}/model'
with mlflow.start_run(run_id=run_ID):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB_SMOTE' already exists. Creating a new version of this model...
2024/08/19 22:06:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_SMOTE, version 2
Created version '2' of model 'XGB_SMOTE'.


## Load the model

In [26]:
model_name = "XGB_SMOTE"
version_name = 1
artifact_name = "model"
# model_uri = f'runs:/d88b94bf2fc347e1a4e3f2849e787c05/model'
# model_uri = f'models:/{model_name}@challenger'
model_uri = f'models:/{model_name}/{version_name}'

print(model_uri)

loaded_model = mlflow.sklearn.load_model(model_uri=model_uri)
preds = loaded_model.predict(X_test)
preds, y_test

models:/XGB_SMOTE/1


(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 

## Transition from dev to production

In [27]:
client = mlflow.MlflowClient()
registry_model_name = "model-in-prod"
client.copy_model_version(src_model_uri=model_uri, dst_name=registry_model_name)

Successfully registered model 'model-in-prod'.
Copied version '1' of model 'XGB_SMOTE' to version '1' of model 'model-in-prod'.


<ModelVersion: aliases=[], creation_timestamp=1724086692739, current_stage='None', description='', last_updated_timestamp=1724086692739, name='model-in-prod', run_id='d88b94bf2fc347e1a4e3f2849e787c05', run_link='', source='models:/XGB_SMOTE/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [29]:
prod_model_uri = f"models:/{registry_model_name}@champion"

prod_model = mlflow.sklearn.load_model(model_uri=prod_model_uri)
pred = prod_model.predict(X_test)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0])